In [61]:
from tqdm import tqdm
import polars as pl

In [51]:
krs = pl.read_parquet("/home/stef/enz_rxn_data/data/processed/pathway/known_reactions.parquet")

In [53]:
krs.filter(pl.col("enzymes").list.len() > 0)

id,smarts,enzymes,reverse,db_ids
i32,str,list[str],i32,list[str]
0,"""**.NC(CCC(=O)NC(CS)C(=O)NCC(=O…","[""Q63688"", ""O75881"", ""Q60991""]",4980,"[""RHEA:16438""]"
1,"""*.*.*.*.*.*.*.*.*.*.CC(C)(COP(…","[""Q03220"", ""Q8QZ73"", … ""O90368""]",7915,"[""RHEA:51350""]"
6,"""*.*.*.*.*.C=CC1=C(C)C(=Cc2[nH]…","[""Q99PW5"", ""O97859"", … ""Q9Y3R4""]",6146,"[""RHEA:37365""]"
7,"""*.*.*.*.*.C=CC1=C(C)C(=O)NC1=C…","[""C8WGQ3"", ""C8WJW0""]",6147,"[""RHEA:37041""]"
8,"""*.*.*.*.*.CC(C)=CCCC(C)=CCCC(C…","[""Q2XNC8"", ""Q91WL5"", … ""P51589""]",9882,"[""RHEA:30976""]"
…,…,…,…,…
29183,"""[N]=O.[N]=O>>N#[N+][O-].O""","[""A0A4V8H042""]",21896,"[""RHEA:10990""]"
29184,"""[N]=O.[O]O>>O=NOO""","[""Q8NBS3"", ""A2AJN7""]",28240,"[""RHEA:51189""]"
29189,"""[O]C1=C(O)C(=O)OC1C(O)CO.[O]C1…","[""Q8K4F5"", ""Q8NFV4"", … ""Q3SZ73""]",28044,"[""RHEA:30268""]"


In [48]:
kes = pl.read_parquet("/home/stef/enz_rxn_data/data/processed/pathway/known_enzymes.parquet")
kes.head()

id,sequence,existence,reviewed,ec,organism,name
str,str,enum,enum,str,str,str
"""A0A009IHW8""","""MSLEQKKGADIISKILQIQNSIGKTTSPST…","""Evidence at protein level""","""reviewed""","""3.2.2.-; 3.2.2.6""","""Acinetobacter baumannii (strai…","""2' cyclic ADP-D-ribose synthas…"
"""A0A059TC02""","""MRSVSGQVVCVTGAGGFIASWLVKILLEKG…","""Evidence at protein level""","""reviewed""","""1.2.1.44""","""Petunia hybrida (Petunia)""","""Cinnamoyl-CoA reductase 1 (Ph-…"
"""A0A061I403""","""MPMASVIAVAEPKWISVWGRFLWLTLLSMA…","""Evidence at protein level""","""reviewed""","""2.7.7.108; 3.1.4.-""","""Cricetulus griseus (Chinese ha…","""Protein adenylyltransferase FI…"
"""A0A067XGX8""","""MALTATATTRGGSALPNSCLQTPKFQSLQK…","""Evidence at transcript level""","""reviewed""","""2.5.1.54""","""Petunia hybrida (Petunia)""","""Phospho-2-dehydro-3-deoxyhepto…"
"""A0A067XH53""","""MALSTNSTTSSLLPKTPLVQQPLLKNASLP…","""Evidence at transcript level""","""reviewed""","""2.5.1.54""","""Petunia hybrida (Petunia)""","""Phospho-2-dehydro-3-deoxyhepto…"


In [49]:
kes.filter(pl.col("existence").is_in(["Evidence at protein level", "Evidence at transcript level"]))

id,sequence,existence,reviewed,ec,organism,name
str,str,enum,enum,str,str,str
"""A0A009IHW8""","""MSLEQKKGADIISKILQIQNSIGKTTSPST…","""Evidence at protein level""","""reviewed""","""3.2.2.-; 3.2.2.6""","""Acinetobacter baumannii (strai…","""2' cyclic ADP-D-ribose synthas…"
"""A0A059TC02""","""MRSVSGQVVCVTGAGGFIASWLVKILLEKG…","""Evidence at protein level""","""reviewed""","""1.2.1.44""","""Petunia hybrida (Petunia)""","""Cinnamoyl-CoA reductase 1 (Ph-…"
"""A0A061I403""","""MPMASVIAVAEPKWISVWGRFLWLTLLSMA…","""Evidence at protein level""","""reviewed""","""2.7.7.108; 3.1.4.-""","""Cricetulus griseus (Chinese ha…","""Protein adenylyltransferase FI…"
"""A0A067XGX8""","""MALTATATTRGGSALPNSCLQTPKFQSLQK…","""Evidence at transcript level""","""reviewed""","""2.5.1.54""","""Petunia hybrida (Petunia)""","""Phospho-2-dehydro-3-deoxyhepto…"
"""A0A067XH53""","""MALSTNSTTSSLLPKTPLVQQPLLKNASLP…","""Evidence at transcript level""","""reviewed""","""2.5.1.54""","""Petunia hybrida (Petunia)""","""Phospho-2-dehydro-3-deoxyhepto…"
…,…,…,…,…,…,…
"""P80580""","""MKLYSFFNXRA""","""Evidence at protein level""","""reviewed""","""5.2.1.4""","""Klebsiella pneumoniae""","""Maleylpyruvate isomerase (EC 5…"
"""P83329""","""XXFENFLNANXAYVNLD""","""Evidence at protein level""","""reviewed""","""4.2.1.1""","""Streptococcus thermophilus""","""Putative carbonic anhydrase (E…"
"""P83855""","""GVYMEIGRCRXEAXRRRKEAV""","""Evidence at protein level""","""reviewed""","""4.6.1.1""","""Mus musculus (Mouse)""","""Putative sperm adenylate cycla…"


In [ ]:
pairs = set()
proper_pairs = set()
evidence_dict = dict(zip(kes["id"], kes["existence"]))
for row in tqdm(krs.iter_rows(named=True), total=len(krs)):
    rxn = row['smarts']
    for enz in row['enzymes']:
        evidence = evidence_dict.get(enz, "Unknown")
        
        if evidence in ["Evidence at protein level", "Evidence at transcript level"]:
            proper_pairs.add((rxn, enz))
        
        pairs.add((rxn, enz))

len(pairs), len(proper_pairs)

100%|██████████| 28142/28142 [00:00<00:00, 45076.05it/s]


(518578, 164968)